<a href="https://colab.research.google.com/github/ali-vayani/PokeGen/blob/main/Datascraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
from bs4 import BeautifulSoup
import os
from PIL import Image, ImageOps
import csv
import base64
import re
import pandas as pd

# Getting all the pokemon links

In [ ]:
response = requests.get('https://pokemondb.net/pokedex/all')
soup = BeautifulSoup(response.text, 'html.parser')
rows = soup.select('table#pokedex tbody tr')
pokemon_links = []
for row in rows:
  link_tag = row.find('a', href=True) # gets links
  if link_tag:
    pokemon_links.append('https://pokemondb.net' + link_tag['href'])

pokemon_links = list(sorted(set(pokemon_links)))
print(len(pokemon_links))


1025


# Scraping text & images + adding everything to CSV

In [ ]:
# some pokemon have multiple forms or have different names in url
special_cases = {
    "Eiscue": [
        'https://img.pokemondb.net/artwork/large/eiscue-noice.jpg',
        'https://img.pokemondb.net/artwork/large/eiscue-ice.jpg'
    ],
    "Enamorus": [
        'https://img.pokemondb.net/artwork/large/enamorus-therian.jpg',
        'https://img.pokemondb.net/artwork/large/enamorus-incarnate.jpg'
    ],
    "Flabébé": ['https://img.pokemondb.net/artwork/large/flabebe.jpg'],
    "Giratina": [
        'https://img.pokemondb.net/artwork/large/giratina-origin.jpg',
        'https://img.pokemondb.net/artwork/large/giratina-altered.jpg'
    ],
    "Lycanroc": [
        'https://img.pokemondb.net/artwork/large/lycanroc-midnight.jpg',
        'https://img.pokemondb.net/artwork/large/lycanroc-midday.jpg'
    ],
    "Morpeko": [
        'https://img.pokemondb.net/artwork/large/morpeko-full-belly.jpg',
        'https://img.pokemondb.net/artwork/large/morpeko-hangry.jpg'
    ],
    "Nidoran♀ (female)": ['https://img.pokemondb.net/artwork/large/nidoran-f.jpg'],
    "Nidoran♂ (male)": ['https://img.pokemondb.net/artwork/large/nidoran-m.jpg'],
    "Oricorio": [
        'https://img.pokemondb.net/artwork/large/oricorio-baile.jpg',
        'https://img.pokemondb.net/artwork/large/oricorio-pom-pom.jpg',
        'https://img.pokemondb.net/artwork/large/oricorio-pau.jpg',
        'https://img.pokemondb.net/artwork/large/oricorio-sensu.jpg'
    ],
    "Shaymin": [
        'https://img.pokemondb.net/artwork/large/shaymin-land.jpg',
        'https://img.pokemondb.net/artwork/large/shaymin-sky.jpg'
    ],
    "Urshifu": ['https://img.pokemondb.net/artwork/large/urshifu-single-strike.jpg'],
    "Wishiwashi": [
        'https://img.pokemondb.net/artwork/large/wishiwashi-solo.jpg',
        'https://img.pokemondb.net/artwork/large/wishiwashi-school.jpg'
    ]
}

In [ ]:
def clean_name(name):
    name = name.replace("♀", "").replace("♂", "")
    name = name.replace(" ", "-")
    name = re.sub(r'[^a-zA-Z0-9-]', '', name)
    return name.lower()


# resize func
def resize_image_to_512(img_path, output_path):
    with Image.open(img_path) as img:
        img.thumbnail((512, 512))
        canvas = Image.new('RGB', (512, 512), (255, 255, 255))
        paste_x = (512 - img.width) // 2
        paste_y = (512 - img.height) // 2
        canvas.paste(img, (paste_x, paste_y))
        canvas.save(output_path)

In [ ]:
def process_pokemon(name, img_url, type1, type2, csvwriter):
    try:
        response = requests.get(img_url)
        response.raise_for_status()

        temp_img_path = f'pokemon_images/{name}_original.jpg'
        final_img_path = f'pokemon_images/{name}.jpg'
        with open(temp_img_path, 'wb') as img_file:
            img_file.write(response.content)
        resize_image_to_512(temp_img_path, final_img_path)
        os.remove(temp_img_path)
        description = image_label(img_url)

        # Write to CSV
        csvwriter.writerow([name, final_img_path, type1, type2, description])
        #print(f"Processed {name} with default URL")
    except requests.exceptions.RequestException:
        # If the default URL fails, try special cases
        if name in special_cases:
            for special_url in special_cases[name]:
                try:
                    response = requests.get(special_url)
                    response.raise_for_status()

                    # save and resize the image
                    temp_img_path = f'pokemon_images/{special_url[40:]}_special_original.jpg'
                    final_img_path = f'pokemon_images/{special_url[40:]}'
                    with open(temp_img_path, 'wb') as img_file:
                        img_file.write(response.content)
                    resize_image_to_512(temp_img_path, final_img_path)
                    os.remove(temp_img_path)
                    description = image_label(special_url)

                    # Write to CSV
                    csvwriter.writerow([name, final_img_path, type1, type2, description])
                    return
                except requests.exceptions.RequestException:
                    continue
        print(f"Failed to process {name}. No valid image found.")

In [ ]:
import csv

os.makedirs('pokemon_images', exist_ok=True)

# CSV file setup
csv_filename = 'pokemon_data.csv'
csv_headers = ['Name', 'Image Path', 'Type 1', 'Type 2', 'Description']
with open(csv_filename, mode='w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(csv_headers)

    # gets info + downloads image + writes to CSV for all pokemon
    for url in pokemon_links:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # 1. name
        name = soup.select_one('h1').text.strip()

        # 2. img urls
        img_url = f"https://img.pokemondb.net/artwork/{clean_name(name)}.jpg"

        # 3. types
        rows = soup.select('tr')
        second_row = rows[1] if len(rows) > 1 else None
        type_links = second_row.select('a') if second_row else None
        type1 = type_links[0].text.strip() if len(type_links) > 0 else None
        type2 = type_links[1].text.strip() if len(type_links) > 1 else None

        process_pokemon(name, img_url, type1, type2, csvwriter)

In [ ]:
from openai import OpenAI
client = OpenAI(
  api_key="***"
)
def image_label(image_url):
  response = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
          {
              "role": "user",
              "content": [
                  {"type": "text", "text": "you're an assistant that creates a label for an image. this label should be written in the form of a text entry for a diffusion model."},
                  {
                      "type": "image_url",
                      "image_url": {
                          "url": image_url,
                      },
                  },
              ],
          }
      ],
      max_tokens=200,
  )
  return response.choices[0].message.content

# idfk anymore

In [1]:
!git init
!git remote add origin https://github.com/ali-vayani/PokeGen.git
!git sparse-checkout init --no-cone
!git sparse-checkout set Data
!git pull origin main

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
remote: Enumerating objects: 1056, done.
remote: Counting objects: 100% (1056/1056), done.
remote: Compressing objects: 100% (1049/1049), done.
remote: Total 1056 (delta 12), reused 1035 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (1056/1056), 17.35 MiB | 29.42 MiB/s, done.
Resolving deltas: 100% (12/12), done.
From https://github.com/ali-vayani/PokeGen
 * branch            main       -> FETCH_HEAD
 * [new branch]      main       -> origin/main


In [45]:
df = pd.read_csv('Data/pokemon_data.csv')
columns_to_keep = ['Image Path', 'Description']
new_df = df[columns_to_keep].copy()

new_df.to_csv('poke_data.csv', index=False)

new_df.head()

,Image Path,Description
0,pokemon_images/Abomasnow.jpg,Frosty creature with white fur and green accen...
1,pokemon_images/Abra.jpg,"A yellow, psychic-type Pokémon with large, exp..."
2,pokemon_images/Absol.jpg,"A sleek, mythical creature resembling a canine..."
3,pokemon_images/Accelgor.jpg,"A colorful, cartoony octopus-like creature wit..."
4,pokemon_images/Aegislash.jpg,"Aegislash, a Pokémon that resembles a medieval..."


# pushing to hugging face

In [6]:
pip install datasets huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [22]:
from huggingface_hub import login
from google.colab import userdata

hf_token = userdata.get('HF_TOKEN')
login(token=hf_token)

In [48]:
new_df.rename(columns={"Image Path": "image", "Description": "description"}, inplace=True)
new_df['image'] = new_df['image'].str.replace("data/pokemon_images/", "", regex=False)
new_df.head()

,image,description
0,Abomasnow.jpg,Frosty creature with white fur and green accen...
1,Abra.jpg,"A yellow, psychic-type Pokémon with large, exp..."
2,Absol.jpg,"A sleek, mythical creature resembling a canine..."
3,Accelgor.jpg,"A colorful, cartoony octopus-like creature wit..."
4,Aegislash.jpg,"Aegislash, a Pokémon that resembles a medieval..."


In [55]:
from datasets import Dataset, Features, Value, Image

features = Features({"image": Image(), "description": Value("string")})
dataset = Dataset.from_pandas(new_df, features=features)

# Push the dataset to Hugging Face
dataset.push_to_hub("pAstaWasTaken/poke-data")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/pAstaWasTaken/poke-data/commit/bf597dd0ecda7822768d9819d48f597b3abf13ff', commit_message='Upload dataset', commit_description='', oid='bf597dd0ecda7822768d9819d48f597b3abf13ff', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/pAstaWasTaken/poke-data', endpoint='https://huggingface.co', repo_type='dataset', repo_id='pAstaWasTaken/poke-data'), pr_revision=None, pr_num=None)